# Stale Signal Risk

Measure how market movement evolves after rebalance while targets are still being scaled.

Uses:
- `targets.csv` for week/day mapping
- `positions.csv` for daily prices/weights
- `signals.csv` for rebalance-day signal magnitude (tier)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


## Data Loading & Adverse Move Setup

Loads targets, positions, and signals data, then joins daily prices onto the target schedule to compute a reference price at the start of each rebalance week. The `adverse_move` column measures how much each stock moved against the intended trade direction during the scaling window, and `tier` segments results by signal strength. The head preview confirms all join columns are populated before analysis proceeds.

## Adverse Move and PnL Profiles During Scaling

These two line charts profile market movement and P&L during the scaling period, broken out by signal tier and indexed by day-in-week. The left chart shows mean adverse price move since rebalance — a rising line means the stock is increasingly moving against the unfilled portion of the order. The right chart shows mean daily net P&L during the scaling window, revealing whether delayed filling is associated with weaker realized returns.

## Weekly Stale Signal Risk Summary Table

This table summarizes stale-signal risk at the week level, reporting average and maximum adverse move along with total net P&L for each rebalance cycle. Weeks with high `max_adverse_move` coupled with low `week_net_pnl` identify periods where slow fill execution genuinely hurt performance. Use this table to correlate stale-signal risk with broader market conditions such as trend reversals or elevated volatility regimes.

In [ ]:
df_targets = read_csv_from_store('wolfpack/targets.csv')
df_positions = read_csv_from_store('wolfpack/positions.csv')
df_signals = read_csv_from_store('wolfpack/signals.csv')

if df_targets is None or df_positions is None:
    raise ValueError('targets.csv and positions.csv are required.')

df_targets['date'] = pd.to_datetime(df_targets['date'])
df_positions['date'] = pd.to_datetime(df_positions['date'])

for col in ['start_w', 'weekly_target_w', 'actual_w']:
    df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce').fillna(0.0)
for col in ['price', 'daily_total_net_pnl']:
    if col in df_positions.columns:
        df_positions[col] = pd.to_numeric(df_positions[col], errors='coerce')

px = (
    df_positions[['date', 'symbol', 'price', 'daily_total_net_pnl']]
    .dropna(subset=['price'])
    .drop_duplicates(['date', 'symbol'])
    .sort_values(['symbol', 'date'])
)

tgt = df_targets[['date', 'week_id', 'symbol', 'start_w', 'weekly_target_w', 'actual_w']].drop_duplicates()
merged = tgt.merge(px, on=['date', 'symbol'], how='left')

# Rebalance-day price reference
rebalance_px = (
    merged.sort_values('date')
          .groupby(['week_id', 'symbol'], as_index=False)
          .first()[['week_id', 'symbol', 'price']]
          .rename(columns={'price': 'rebalance_price'})
)
merged = merged.merge(rebalance_px, on=['week_id', 'symbol'], how='left')

merged['day_in_week'] = merged.sort_values('date').groupby(['week_id', 'symbol']).cumcount()
merged['signal_direction'] = np.sign(merged['weekly_target_w']).replace(0, np.nan)
merged['return_since_rebalance'] = (merged['price'] / merged['rebalance_price']) - 1.0
merged['adverse_move'] = -merged['signal_direction'] * merged['return_since_rebalance']

if df_signals is not None:
    df_signals['date'] = pd.to_datetime(df_signals['date'])
    df_signals['week_id'] = df_signals['date'].dt.strftime('%Y-%m-%d')
    df_signals['mag_abs'] = pd.to_numeric(df_signals['magnitude'], errors='coerce').fillna(0.0).abs()
    sig = df_signals[['week_id', 'symbol', 'mag_abs']].drop_duplicates(['week_id', 'symbol'])
    merged = merged.merge(sig, on=['week_id', 'symbol'], how='left')
else:
    merged['mag_abs'] = np.nan

def tier(m):
    if pd.isna(m):
        return 'unknown'
    if m >= 0.7:
        return 'strong'
    if m >= 0.3:
        return 'moderate'
    return 'weak'

merged['tier'] = merged['mag_abs'].apply(tier)
display(merged.head())


In [ ]:
profile = (
    merged.groupby(['tier', 'day_in_week'], as_index=False)
          .agg(
              mean_adverse_move=('adverse_move', 'mean'),
              median_adverse_move=('adverse_move', 'median'),
              mean_daily_net_pnl=('daily_total_net_pnl', 'mean')
          )
)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

sns.lineplot(data=profile, x='day_in_week', y='mean_adverse_move', hue='tier', marker='o', ax=axes[0])
axes[0].axhline(0, color='black', linewidth=1)
axes[0].set_title('Mean Adverse Move Since Rebalance')
axes[0].set_ylabel('Adverse move (fraction)')
axes[0].set_xlabel('Day-in-week')
axes[0].grid(alpha=0.3)

sns.lineplot(data=profile, x='day_in_week', y='mean_daily_net_pnl', hue='tier', marker='o', ax=axes[1])
axes[1].axhline(0, color='black', linewidth=1)
axes[1].set_title('Mean Daily Net PnL During Scaling')
axes[1].set_ylabel('Daily net PnL ($)')
axes[1].set_xlabel('Day-in-week')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
week_risk = (
    merged.groupby('week_id', as_index=False)
          .agg(
              avg_adverse_move=('adverse_move', 'mean'),
              max_adverse_move=('adverse_move', 'max'),
              week_net_pnl=('daily_total_net_pnl', 'sum')
          )
          .sort_values('week_id')
)
display(week_risk.tail(15))
